In [ ]:
import yaml
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from SimBMVtool import SimBMVtool as sim

path_config = './config_simu.yaml'

# Create temporary config file for notebook tests
path_tmp_config = '../outputs_dev/tmp_config_simu.yaml'
with open(path_config, 'r') as file: tmp_config = yaml.safe_load(file)

In [ ]:
accmodel = sim.BaseSimBMVCreator()
accmodel.load_true_background_irfs(path_config) 
accmodel.load_output_background_irfs()

if accmodel.out_collection: 
    accmodel.bkg_output_irf_collection[1].plot_at_energy(["1 TeV"],figsize=(3,3))
    accmodel.bkg_output_irf_collection[1].peek()
else: 
    accmodel.bkg_output_irf.plot_at_energy(["1 TeV"],figsize=(3,3))
    accmodel.bkg_output_irf.peek()

In [ ]:
plot_data = True
if plot_data:
    # TO-DO: re-implement zenith binned plotting within plot_method
    icos='all'
    accmodel.plot_model(data='acceptance', irf='true', residuals='none',title="Acceptance: true", fig_save_path=f"{accmodel.plots_dir}/acceptance_true_cosbin_{icos}.png")
    accmodel.plot_model(data='acceptance', irf='output', residuals='none',title="Acceptance: output", fig_save_path=f"{accmodel.plots_dir}/acceptance_out_cosbin_{icos}.png")
    accmodel.plot_model(data='acceptance', irf='both', residuals='diff/true',title="Acceptance: residuals", fig_save_path=f"{accmodel.plots_dir}/acceptance_residuals_cosbin_{icos}.png", plot_hist=True)
    accmodel.plot_model(data='acceptance', irf='both', residuals='diff/sqrt(true)', title="Acceptance: residuals", fig_save_path=f"{accmodel.plots_dir}/acceptance_residuals_sqrt_cosbin_{icos}.png")

In [ ]:
plot_profiles = True
if plot_profiles:
    accmodel.plot_profile(irf='output', profile='both', stat='sum', bias=False, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{accmodel.plots_dir}/profile_stat_sum_cosbin_{icos}.png")
    accmodel.plot_profile(irf='both', profile='both', stat='mean', bias=True, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{accmodel.plots_dir}/profile_bias_mean_cosbin_{icos}.png")
    accmodel.plot_profile(irf='both', profile='both', stat='std', bias=True, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{accmodel.plots_dir}/profile_bias_std_cosbin_{icos}.png")

In [ ]:
# Check what your output skymaps should look like
plot_skymaps  = True
if plot_skymaps:
    accmodel.load_observation_collection() # Load simulated data with true background model
    accmodel.plot_skymaps('ring')           # Map spatial axis by default use true irf axes info, all energies are stacked
    accmodel.plot_skymaps('FoV')

In [ ]:
# Load simulated data with output model, uses index tables based on naming scheme following config file info
accmodel.load_observation_collection(from_index=True)               

# Change skymap parameters directly within the class object
# For instance to plot for each energy bins with offset safe mask depending on the output model
# Here you can see that the lack of statistics end up giving empty bins at large offset for the larger energies

# To get results similar to the previous cell, you would need more simulated livetime and a model performed with a larger offset_max
# due to the very large offset of the source in HESS's very large field of view

for (e_min_map,e_max_map), offset_max_dataset in zip(accmodel.Ebin_tuples, [1.5, 1.3, 1.1]):
    offset_max_map = 2
    accmodel.axis_info_dataset = [e_min_map * u.TeV, e_max_map * u.TeV, offset_max_dataset * u.deg, accmodel.nbin_offset_irf]
    accmodel.axis_info_map = [e_min_map * u.TeV, e_max_map * u.TeV, offset_max_map * u.deg, accmodel.nbin_offset_irf]
    accmodel.plot_skymaps('ring')

In [ ]:
e_min_map, e_max_map = (0.5 * u.TeV, 1.4 * u.TeV) # Here we confide in the first energy been where the model has no empty bin
offset_max_dataset, offset_max_map = (1.5, 1.5) * u.deg # We can use the full offset range of the model in this energy bin

# If you want to change the exclusion region radius, you can use a temporary config file to reinitialize the exclusion region
i_circle = 1 # First circle is center on the source, you can add other circles in config file if you want to mask a star
exclusion_radius_arr = [0.3, 0.35, 0.4]

for i_radbin, radius in enumerate(exclusion_radius_arr):
    tmp_config["source"]["exclusion_region"]["n_circles"][f"circle_{i_circle}"]["radius"] = radius
    tmp_config["source"]["exclusion_region"]["shape"] = "n_circles" if radius > 0 else "noexclusion"
    with open(path_tmp_config, 'w') as file: yaml.dump(tmp_config, file)
    
    accmodel.init_config(path_tmp_config)
    accmodel.axis_info_dataset = [e_min_map, e_max_map, offset_max_dataset, accmodel.nbin_offset_irf]
    accmodel.axis_info_map = [e_min_map, e_max_map, offset_max_map, accmodel.nbin_offset_irf]

    accmodel.plot_skymaps('ring')

In [ ]:
# Plotting everything takes some time, and you may need to look only at one map
stacked_dataset = accmodel.get_stacked_dataset(bkg_method='ring', axis_info=accmodel.axis_info_dataset)
skymaps = sim.get_skymaps_dict(stacked_dataset, accmodel.exclude_regions, accmodel.correlation_radius, accmodel.correlate_off, 'all')
for key in skymaps.keys(): print(skymaps[key])

sim.plot_skymap_from_dict(skymaps, 'counts', figsize=(4,4))

# Crop the map
sim.plot_skymap_from_dict(skymaps, 'significance_off', crop_width=1.2*u.deg)

In [ ]:
# Change any parameter you need for the modeling in the config file
with open(path_config, 'r') as file: tmp_config = yaml.safe_load(file)
tmp_config["acceptance"]["method"] = 'fit'
with open(path_tmp_config, 'w') as file: yaml.dump(tmp_config, file)

# Uncomment if you haven't already done the modeling
# accmodel.do_acceptance_modelisation(path_tmp_config) 
accmodel.load_output_background_irfs(path_tmp_config)
accmodel.load_observation_collection(from_index=True) # Uses index tables based on naming scheme following config file info

plot_skymaps  = True
if plot_skymaps:
    accmodel.axis_info_dataset = [accmodel.e_min, accmodel.e_max, accmodel.size_fov_acc, accmodel.nbin_offset_irf]
    accmodel.axis_info_map = [accmodel.e_min, accmodel.e_max, accmodel.size_fov_acc, accmodel.nbin_offset_irf]
    accmodel.plot_skymaps('ring')
    accmodel.plot_skymaps('FoV')